## Titanic Survived Prediction using Decision Tree

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler


Data Loading , Feature Selection and Preprocessing

In [5]:
data= pd.read_csv('/content/drive/MyDrive/Titanic-Dataset.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [9]:
data.dropna(inplace=True)


In [11]:

mapping= { 'C':0,'S':1,'Q':2}
data['Embarked']= data['Embarked'].map(mapping)
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,1
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,1
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,1
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,1
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,1
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,1
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,1


In [13]:
data= data.join(pd.get_dummies(data['Sex']).astype(int)).drop('Sex',axis=1)
data

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,female,male
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,1,1,0
6,7,0,1,"McCarthy, Mr. Timothy J",54.0,0,0,17463,51.8625,E46,1,0,1
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",4.0,1,1,PP 9549,16.7000,G6,1,1,0
11,12,1,1,"Bonnell, Miss. Elizabeth",58.0,0,0,113783,26.5500,C103,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",47.0,1,1,11751,52.5542,D35,1,1,0
872,873,0,1,"Carlsson, Mr. Frans Olof",33.0,0,0,695,5.0000,B51 B53 B55,1,0,1
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",56.0,0,1,11767,83.1583,C50,0,1,0
887,888,1,1,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,B42,1,1,0


In [20]:
data.drop('male',axis=1,inplace=True)
data

data

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,female
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,1,1
6,7,0,1,"McCarthy, Mr. Timothy J",54.0,0,0,17463,51.8625,E46,1,0
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",4.0,1,1,PP 9549,16.7000,G6,1,1
11,12,1,1,"Bonnell, Miss. Elizabeth",58.0,0,0,113783,26.5500,C103,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",47.0,1,1,11751,52.5542,D35,1,1
872,873,0,1,"Carlsson, Mr. Frans Olof",33.0,0,0,695,5.0000,B51 B53 B55,1,0
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",56.0,0,1,11767,83.1583,C50,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,B42,1,1


In [23]:
#dropping the name
data.drop('Name',axis=1,inplace =True)
data


,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,female
1,1,1,38.0,1,0,PC 17599,71.2833,C85,0,1
3,1,1,35.0,1,0,113803,53.1000,C123,1,1
6,0,1,54.0,0,0,17463,51.8625,E46,1,0
10,1,3,4.0,1,1,PP 9549,16.7000,G6,1,1
11,1,1,58.0,0,0,113783,26.5500,C103,1,1
...,...,...,...,...,...,...,...,...,...,...
871,1,1,47.0,1,1,11751,52.5542,D35,1,1
872,0,1,33.0,0,0,695,5.0000,B51 B53 B55,1,0
879,1,1,56.0,0,1,11767,83.1583,C50,0,1
887,1,1,19.0,0,0,112053,30.0000,B42,1,1


In [25]:
data=data.join(pd.get_dummies(data['Cabin']).astype(int)).drop('Cabin',axis=1)
data

,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Embarked,female,A10,A16,A20,A23,A24,A26,A31,A34,A36,A5,A6,A7,B101,B18,B19,B20,B22,B3,B30,B35,B37,B38,B39,B4,B41,B42,B49,B5,B50,B51 B53 B55,B57 B59 B63 B66,B58 B60,B69,B71,B73,B77,B79,B80,B82 B84,B86,B94,...,D15,D17,D19,D20,D26,D28,D30,D33,D35,D36,D37,D46,D47,D48,D49,D50,D56,D6,D7,D9,E10,E101,E12,E121,E17,E24,E25,E31,E33,E34,E36,E38,E40,E44,E46,E49,E50,E58,E63,E67,E68,E77,E8,F G63,F G73,F2,F33,F4,G6,T
1,1,1,38.0,1,0,PC 17599,71.2833,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,35.0,1,0,113803,53.1000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,1,54.0,0,0,17463,51.8625,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,1,3,4.0,1,1,PP 9549,16.7000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
11,1,1,58.0,0,0,113783,26.5500,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,1,1,47.0,1,1,11751,52.5542,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
872,0,1,33.0,0,0,695,5.0000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
879,1,1,56.0,0,1,11767,83.1583,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
887,1,1,19.0,0,0,112053,30.0000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
data=data.join(pd.get_dummies(data['Ticket']).astype(int)).drop('Ticket',axis=1)
data

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,female,A10,A16,A20,A23,A24,A26,A31,A34,A36,A5,A6,A7,B101,B18,B19,B20,B22,B3,B30,B35,B37,B38,B39,B4,B41,B42,B49,B5,B50,B51 B53 B55,B57 B59 B63 B66,B58 B60,B69,B71,B73,B77,B79,B80,B82 B84,B86,B94,B96 B98,...,348121,348123,348124,35273,35281,36928,36947,36963,36967,36973,392096,5727,695,C.A. 29395,C.A. 34260,F.C. 12750,PC 17473,PC 17474,PC 17475,PC 17476,PC 17477,PC 17482,PC 17485,PC 17558,PC 17569,PC 17572,PC 17582,PC 17590,PC 17592,PC 17593,PC 17595,PC 17596,PC 17599,PC 17608,PC 17610,PC 17754,PC 17755,PC 17756,PC 17757,PC 17758,PC 17759,PC 17760,PC 17761,PP 9549,S.O./P.P. 3,SC/AH Basle 541,SC/Paris 2163,SOTON/O.Q. 392078,W.E.P. 5734,WE/P 5735
1,1,1,38.0,1,0,71.2833,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,35.0,1,0,53.1000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,1,54.0,0,0,51.8625,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,1,3,4.0,1,1,16.7000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
11,1,1,58.0,0,0,26.5500,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,1,1,47.0,1,1,52.5542,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
872,0,1,33.0,0,0,5.0000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
879,1,1,56.0,0,1,83.1583,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
887,1,1,19.0,0,0,30.0000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
data.corr()['Survived']

,Survived
Survived,1.000000
Pclass,-0.034542
Age,-0.254085
SibSp,0.106346
Parch,0.023582
...,...
SC/AH Basle 541,0.051771
SC/Paris 2163,-0.106131
SOTON/O.Q. 392078,0.051771
W.E.P. 5734,-0.106131


Training using Decision Tree Classifier.

In [34]:
#Splitting The data
x_train,x_test,y_train,y_test= train_test_split(data.drop('Survived',axis=1),data['Survived'],test_size=0.3,random_state=42)

In [35]:
#Scalling the Data
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [36]:
#Decision Tree Model training
model_decision =DecisionTreeClassifier()
model_decision.fit(x_train,y_train)

DecisionTreeClassifier()

In [37]:
#Accuracy Score
from sklearn.metrics import accuracy_score
prediction=model_decision.predict(x_test)
accuracy_score(y_test,prediction)

0.7272727272727273

In [38]:
#Consufion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,prediction)

array([[15,  5],
       [10, 25]])

In [39]:
#auc and roc
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,prediction)

0.7321428571428572

In [40]:
#f1
from sklearn.metrics import f1_score
f1_score(y_test,prediction)

0.7692307692307693

In [41]:
#recall
from sklearn.metrics import recall_score
recall_score(y_test,prediction)

0.7142857142857143

## Titanic Using Regression Model and Naive Bias Classifier

In [42]:
x_train,x_test,y_train,y_test= train_test_split(data.drop('Survived',axis=1),data['Survived'],test_size=0.3,random_state=42)


In [43]:
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [44]:
#Using GaussianNb for Naive Bias Classification for the same data
from sklearn.naive_bayes import GaussianNB
model_naive=GaussianNB()
model_naive.fit(x_train,y_train)

GaussianNB()

In [50]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
predict_naive=model_naive.predict(x_test)
accuracy_score(y_test,predict_naive)
print(classification_report(y_test,predict_naive))
print(confusion_matrix(y_test,predict_naive))

              precision    recall  f1-score   support

           0       0.43      0.80      0.56        20
           1       0.78      0.40      0.53        35

    accuracy                           0.55        55
   macro avg       0.61      0.60      0.54        55
weighted avg       0.65      0.55      0.54        55

[[16  4]
 [21 14]]


In [52]:
from sklearn.metrics import accuracy_score
predict_naive=model_naive.predict(x_test)
accuracy_score(y_test,predict_naive)

0.5454545454545454

## Using Logistic

In [56]:
from sklearn.linear_model import LogisticRegression
model_logistic=LogisticRegression()
model_logistic.fit(x_train,y_train)



LogisticRegression()

In [57]:
from sklearn.metrics import accuracy_score
y_pred_logistic=model_logistic.predict(x_test)
accuracy_score(y_test,y_pred_logistic)

0.7454545454545455

In [58]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(classification_report(y_test,y_pred_logistic))
print(confusion_matrix(y_test,y_pred_logistic))

              precision    recall  f1-score   support

           0       0.67      0.60      0.63        20
           1       0.78      0.83      0.81        35

    accuracy                           0.75        55
   macro avg       0.73      0.71      0.72        55
weighted avg       0.74      0.75      0.74        55

[[12  8]
 [ 6 29]]
